In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import datajoint as dj
import os
os.chdir('..')
import pathlib
import re
import inspect
import importlib

In [28]:
schema = dj.schema()

In [33]:
_linking_module = None


In [34]:
import djarchive_client
client = djarchive_client.client()

In [35]:
list(client.datasets())


['t',
 'workflow-array-ephys-benchmark',
 'workflow-calcium-imaging-test-set',
 'workflow-trial']

In [36]:
list(client.revisions())                


[('t', '1'),
 ('workflow-array-ephys-benchmark', 'v1'),
 ('workflow-calcium-imaging-test-set', '0_1_0a2'),
 ('workflow-trial', '0.0.0b1')]

In [37]:
from workflow_array_ephys import version
revision = version.__version__.replace('.', '_')
revision

'0_1_0a4'

In [4]:
def create_cambridge_probe(probe_type='cambridge M2'):
    """
    Create `ProbeType` and `Electrode` for cambridge probes:
    1.0 (M2)
    For electrode location, the (0, 0) is the
    bottom left corner of the probe (ignore the tip portion)
    Electrode numbering is 1-indexing
    """

    def build_electrodes(site_count, col_spacing, row_spacing,
                        white_spacing, col_count=2,
                        shank_count=1, shank_spacing=31):
        """
        :param site_count: site count per shank
        :param col_spacing: (um) horrizontal spacing between sites
        :param row_spacing: (um) vertical spacing between columns
        :param white_spacing: (um) offset spacing
        :param col_count: number of column per shank
        :param shank_count: number of shank
        :param shank_spacing: spacing between shanks
        :return:
        """
        row_count = int(site_count / col_count)
        x_coords = np.tile([0, 0 + col_spacing], row_count)
        x_white_spaces = np.tile([white_spacing, white_spacing, 0, 0], int(row_count / 2))

        x_coords = np.zeros(64)
        y_coords = np.arange(0,31*64,31)

        #x_coords = x_coords + x_white_spaces
        #y_coords = np.repeat(np.arange(row_count) * row_spacing, 1)

        shank_cols = np.tile([0, 1], row_count)
        shank_rows = np.repeat(range(row_count), 1)

        npx_electrodes = []
        for shank_no in range(shank_count):
            npx_electrodes.extend([{'electrode': (site_count * shank_no) + e_id,
                                    'shank': shank_no,
                                    'shank_col': c_id,
                                    'shank_row': r_id,
                                    'x_coord': x + (shank_no * shank_spacing),
                                    'y_coord': y}
                                for e_id, (c_id, r_id, x, y) in enumerate(
                zip(shank_cols, shank_rows, x_coords, y_coords))])

        return npx_electrodes

    # ---- 1.0 3A ----
    if probe_type == 'cambridge M2':
        electrodes = build_electrodes(site_count=64, col_spacing=0, row_spacing=31,
                                    white_spacing=0, col_count=1)

    return electrodes

In [5]:
electrodes = create_cambridge_probe(probe_type='cambridge M2')
electrodes

[{'electrode': 0,
  'shank': 0,
  'shank_col': 0,
  'shank_row': 0,
  'x_coord': 0.0,
  'y_coord': 0},
 {'electrode': 1,
  'shank': 0,
  'shank_col': 1,
  'shank_row': 1,
  'x_coord': 0.0,
  'y_coord': 31},
 {'electrode': 2,
  'shank': 0,
  'shank_col': 0,
  'shank_row': 2,
  'x_coord': 0.0,
  'y_coord': 62},
 {'electrode': 3,
  'shank': 0,
  'shank_col': 1,
  'shank_row': 3,
  'x_coord': 0.0,
  'y_coord': 93},
 {'electrode': 4,
  'shank': 0,
  'shank_col': 0,
  'shank_row': 4,
  'x_coord': 0.0,
  'y_coord': 124},
 {'electrode': 5,
  'shank': 0,
  'shank_col': 1,
  'shank_row': 5,
  'x_coord': 0.0,
  'y_coord': 155},
 {'electrode': 6,
  'shank': 0,
  'shank_col': 0,
  'shank_row': 6,
  'x_coord': 0.0,
  'y_coord': 186},
 {'electrode': 7,
  'shank': 0,
  'shank_col': 1,
  'shank_row': 7,
  'x_coord': 0.0,
  'y_coord': 217},
 {'electrode': 8,
  'shank': 0,
  'shank_col': 0,
  'shank_row': 8,
  'x_coord': 0.0,
  'y_coord': 248},
 {'electrode': 9,
  'shank': 0,
  'shank_col': 1,
  'shank_r

In [1]:
import mysql.connector

In [29]:
@schema
class ProbeInsertion(dj.Manual):
    definition = """
    # Probe insertion implanted into an animal for a given session.
    -> Session
    insertion_number: tinyint unsigned
    ---
    -> probe.Probe
    """


@schema
class InsertionLocation(dj.Manual):
    definition = """
    # Brain Location of a given probe insertion.
    -> ProbeInsertion
    ---
    -> SkullReference
    ap_location: decimal(6, 2) # (um) anterior-posterior; ref is 0; more anterior is more positive
    ml_location: decimal(6, 2) # (um) medial axis; ref is 0 ; more right is more positive
    depth:       decimal(6, 2) # (um) manipulator depth relative to surface of the brain (0); more ventral is more negative
    theta=null:  decimal(5, 2) # (deg) - elevation - rotation about the ml-axis [0, 180] - w.r.t the z+ axis
    phi=null:    decimal(5, 2) # (deg) - azimuth - rotation about the dv-axis [0, 360] - w.r.t the x+ axis
    beta=null:   decimal(5, 2) # (deg) rotation about the shank of the probe [-180, 180] - clockwise is increasing in degree - 0 is the probe-front facing anterior
    """

__main__.ProbeInsertion